Попробуем применить CUPED на практике на синтетических данных (на реальных, конечно, эффект будет не таким большим).

Воспользуйтесь кодом для генерации данных:

```python
users_num = 10000

df = pd.DataFrame()
df['user'] = range(users_num)
df['group'] = np.random.rand(users_num) < 0.5

df['user_mean'] = np.random.lognormal(mean=np.log(1000), sigma=0.5, size=users_num)
df['cost_before'] = np.abs(
    df['user_mean'] + np.random.normal(0, 100, size=users_num)
)
df['cost'] = np.abs(
    df['user_mean'] + np.random.normal(50, 100, size=users_num)
)
```

В нём мы генерируем нашу метрику «трат» для 2-х групп пользователей так, что исходная привычка распределена логнормально, а в наших периодах до и во время эксперимента с некоторым нормальным отклонением от привычки каждого пользователя. При этом в тестовом периоде люди тратят больше.

Примените к этим данным CUPED и посчитайте, во сколько раз уменьшилась дисперсия в вашей выборке.

In [19]:
import pandas as pd
import numpy as np
import hashlib

In [3]:
users_num = 10000

df = pd.DataFrame()
df['user'] = range(users_num)
df['group'] = np.random.rand(users_num) < 0.5

df['user_mean'] = np.random.lognormal(mean=np.log(1000), sigma=0.5, size=users_num)
df['cost_before'] = np.abs(
    df['user_mean'] + np.random.normal(0, 100, size=users_num)
)
df['cost'] = np.abs(
    df['user_mean'] + np.random.normal(50, 100, size=users_num)
)

In [8]:
theta = np.cov(df.cost_before, df.cost)[0,1] / np.var(df.cost_before)

In [10]:
df['cost_cuped'] = (df['cost'] - theta * (df.cost_before - np.mean(df.cost_before)))

In [12]:
np.var(df['cost_cuped'])

19537.237703629653

In [15]:
np.var(df['cost'])

370644.0578628819

In [16]:
np.var(df['cost']) / np.var(df['cost_cuped'])

18.971159766051432

Давайте практиковаться в бакетном тестирование, а заодно проверим, как будет различаться дисперсия CTR в бакете, если считать её двумя способами. Сгенерируйте данные следующим кодом:

```python
np.random.seed(6)

users_num = 10000
mean_user_ctr = 0.2
beta = 20
alpha = mean_user_ctr * beta / (1 - mean_user_ctr)

df = pd.DataFrame()
df['user'] = range(users_num)
df['group'] = np.random.rand(users_num) < 0.5

df['base_user_ctr'] = np.random.beta(alpha, beta, size=users_num)
df['views'] = np.random.lognormal(mean=1, sigma=1, size=users_num).astype(int) + 1
df['clicks'] = np.random.binomial(df['views'], df['base_user_ctr'])
```

Сделайте переход к бакетам (возьмём 100 штук), с помощью md5 кэша с солью, применённой к айдишнику пользователя (от соли результат сильно различаться не будет, но можете взять 'my_salt').

Далее посчитайте два вида CTR: групповой (на основе количества кликов и просмотров в бакете) и обычный средний CTR пользователей в бакете (это нужно, чтобы мы сравнивали выборки одинаковых длин). Посчитайте среднеквадратичное отклонение этих CTR и сравните по одной из групп значения между этими способами.

In [18]:
np.random.seed(6)

users_num = 10000
mean_user_ctr = 0.2
beta = 20
alpha = mean_user_ctr * beta / (1 - mean_user_ctr)

df = pd.DataFrame()
df['user'] = range(users_num)
df['group'] = np.random.rand(users_num) < 0.5

df['base_user_ctr'] = np.random.beta(alpha, beta, size=users_num)
df['views'] = np.random.lognormal(mean=1, sigma=1, size=users_num).astype(int) + 1
df['clicks'] = np.random.binomial(df['views'], df['base_user_ctr'])

In [21]:
df['bucket'] = df['user'].apply(lambda x: int(hashlib.md5((str(x) + 'my_salt').encode()).hexdigest(), 16) % 100)

In [36]:
group_ctr = df.groupby(['bucket', 'group'])['clicks', 'views'].sum()

/tmp/ipykernel_62/353203589.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  group_ctr = df.groupby(['bucket', 'group'])['clicks', 'views'].sum()


In [37]:
group_ctr['ctr'] = group_ctr.clicks / group_ctr.views 

In [38]:
group_ctr.ctr.std()

0.027449422546377524

In [31]:
df['ctr'] = group_ctr.clicks / group_ctr.views

In [39]:
df.groupby(['bucket', 'group'])['ctr'].mean().std()

0.01734820521879646